<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3 Interlude: Chisel Standard Library
**Prev: [Generators: Collections](3.2_collections.ipynb)**<br>
**Next: [Higher-Order Functions](3.3_higher-order_functions.ipynb)**

## Motivation
Chisel is all about re-use, so it only makes sense to provide a standard library of interfaces (encouraging interoperability of RTL) and generators for commonly-used hardware blocks.

## Setup

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Compiling Main.sc
Downloaded https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.11/3.0.0/chisel3_2.11-3.0.0.pom.sha1
Downloaded https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.11/3.0.0/chisel3_2.11-3.0.0.pom
Downloaded https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.11/3.0.0/
Downloaded https://repo1.maven.org/maven2/com/github/scopt/scopt_2.11/3.6.0/scopt_2.11-3.6.0.pom
Downloaded https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl_2.11/1.0.0/firrtl_2.11-1.0.0.pom
Downloaded https://repo1.maven.org/maven2/com/github/scopt/scopt_2.11/3.6.0/scopt_2.11-3.6.0.pom.sha1
Downloaded https://repo1.maven.org/maven2/com/github/scopt/scopt_2.11/3.6.0/
Downloaded https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl_2.11/1.0.0/firrtl_2.11-1.0.0.pom.sha1
Downloaded https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl_2.11/1.0.0/
Downloaded https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.7/antlr4-runtime-4.7.pom
Downloaded https://repo1.maven.org/maven2/org/antlr

path: String = "/home/ducky/git/generator-bootcamp/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

---
# The Cheatsheet
The [Chisel3 cheatsheet](https://chisel.eecs.berkeley.edu/doc/chisel-cheatsheet3.pdf) contains a summary of all the major hardware construction APIs, including some of the standard library utilities that we'll introduce below.

# Decoupled, a standard Ready-Valid Interface
One of the commonly used interfaces provided by Chisel is `DecoupledIO`, providing a ready-valid interface for transferring data. The idea is that the sink drives the `bits` signal with the data to be transferred and the `valid` signal when there is data to be transferred. The sink drives the `ready` signal when it is ready to accept data, and data is considered transferred when both `ready` and `valid` are asserted on a cycle.

This provides a flow control mechanism in both directions for data transfer, including a backpressure mechanism.

Note: `ready` and `valid` should not be combinationally coupled, otherwise this may result in unsynthesizable combinational loops. `ready` should only be dependent on whether the sink is able to receive data, and `valid` should only be dependent on whether the source has data. Only after the transaction (on the next clock cycle) should the values update.

Any Chisel data can be wrapped in a `DecoupledIO` (used as the `bits` field) as follows:

```scala
val myChiselData = UInt(8.W)
// or any Chisel data type, such as Bool(), SInt(...), or even custom Bundles
val myDecoupled = Decoupled(myChiselData)
```

The above creates a new `DecoupledIO` `Bundle` with fields
- `valid`: Input(Bool)
- `ready`: Input(Bool)
- `bits`: Output(UInt(8.W))
___

The rest of the section will be structured somewhat differently from the ones before: instead of giving you coding exercises, we're going to give some code examples and testcases that print the circuit state. Try to predict what will be printed before just running the tests.

## Queues

`Queue` creates a FIFO (first-in, first-out) queue with Decoupled interfaces on both sides, allowing backpressure. Both the data type and number of elements are configurable.

In [12]:
Driver(() => new Module {
    // Example circuit using a Queue
    val io = IO(new Bundle {
      val in = Flipped(Decoupled(UInt(8.W)))
      val out = Decoupled(UInt(8.W))
    })
    val queue = Queue(io.in, 2)  // 2-element queue
    io.out <> queue
  }) { c => new PeekPokeTester(c) {
    // Example testsequence showing the use and behavior of Queue
    poke(c.io.out.ready, 0)
    poke(c.io.in.valid, 1)  // Enqueue an element
    poke(c.io.in.bits, 42)
    println(s"Starting: io.in: ready=${peek(c.io.in.ready)}; io.out: valid=${peek(c.io.out.valid)}, bits=${peek(c.io.out.bits)}")
    step(1)
  
    poke(c.io.in.valid, 1)  // Enqueue another element
    poke(c.io.in.bits, 43)
    // What do you think io.out.valid and io.out.bits will be?
    println(s"After first enqueue: io.in: ready=${peek(c.io.in.ready)}; io.out: valid=${peek(c.io.out.valid)}, bits=${peek(c.io.out.bits)}")
    step(1)
  
    poke(c.io.in.valid, 1)  // Read a element, attempt to enqueue
    poke(c.io.in.bits, 44)
    poke(c.io.out.ready, 1)
    // What do you think io.in.ready will be, and will this enqueue succeed, and what will be read?
    println(s"On first read: io.in: ready=${peek(c.io.in.ready)}; io.out: valid=${peek(c.io.out.valid)}, bits=${peek(c.io.out.bits)}")
    step(1)
  
    poke(c.io.in.valid, 0)  // Read elements out
    poke(c.io.out.ready, 1)
    // What do you think will be read here?
    println(s"On second read: io.in: ready=${peek(c.io.in.ready)}; io.out: valid=${peek(c.io.out.valid)}, bits=${peek(c.io.out.bits)}")
    step(1)
  
    // Will a third read produce anything?
    println(s"On third read: io.in: ready=${peek(c.io.in.ready)}; io.out: valid=${peek(c.io.out.valid)}, bits=${peek(c.io.out.bits)}")
    step(1)
} }

[info] [0.000] Elaborating design...
[info] [0.007] Done elaborating.
Total FIRRTL Compile Time: 49.5 ms
Total FIRRTL Compile Time: 32.0 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1512596402774
[info] [0.001] Starting: io.in: ready=1; io.out: valid=0, bits=36
[info] [0.003] After first enqueue: io.in: ready=1; io.out: valid=1, bits=42
[info] [0.006] On first read: io.in: ready=0; io.out: valid=1, bits=42
[info] [0.008] On second read: io.in: ready=1; io.out: valid=1, bits=43
[info] [0.009] On third read: io.in: ready=1; io.out: valid=0, bits=42
test cmd11WrapperHelperanonfun1anon1 Success: 0 tests passed in 10 cycles taking 0.016507 seconds
[info] [0.010] RAN 5 CYCLES PASSED


res11: Boolean = true

## Arbiters
Arbiters routes data fron _n_ `DecoupledIO` sources to one `DecoupledIO` sink, given a prioritization.
There are two types included in Chisel:
- `Arbiter`: prioritizes lower-index producers
- `RRArbiter`: runs in round-robin order

Note that Arbiter routing is implemented in combinational logic.

The below example will demonstrate the use of the priority arbiter (which you will also implement in the next section):

In [21]:
Driver(() => new Module {
    // Example circuit using a priority arbiter
    val io = IO(new Bundle {
      val in = Flipped(Vec(2, Decoupled(UInt(8.W))))
      val out = Decoupled(UInt(8.W))
    })
    // Arbiter doesn't have a convenience constructor, so it's built like any Module
    val arbiter = Module(new Arbiter(UInt(8.W), 2))  // 2 to 1 Priority Arbiter
    arbiter.io.in <> io.in
    io.out <> arbiter.io.out
  }) { c => new PeekPokeTester(c) {
    poke(c.io.in(0).valid, 0)
    poke(c.io.in(1).valid, 0)
    println(s"Start: in(0).ready=${peek(c.io.in(0).ready)}, in(1).ready=${peek(c.io.in(1).ready)}, out.valid=${peek(c.io.out.valid)}, out.bits=${peek(c.io.out.bits)}")
    poke(c.io.in(1).valid, 1)  // Valid input 1
    poke(c.io.in(1).bits, 42)
    // What do you think the output will be?
    println(s"valid input 1: in(0).ready=${peek(c.io.in(0).ready)}, in(1).ready=${peek(c.io.in(1).ready)}, out.valid=${peek(c.io.out.valid)}, out.bits=${peek(c.io.out.bits)}")
    poke(c.io.in(0).valid, 1)  // Valid inputs 0 and 1
    poke(c.io.in(0).bits, 43)
    // What do you think the output will be? Which inputs will be ready?
    println(s"valid inputs 0 and 1: in(0).ready=${peek(c.io.in(0).ready)}, in(1).ready=${peek(c.io.in(1).ready)}, out.valid=${peek(c.io.out.valid)}, out.bits=${peek(c.io.out.bits)}")
    poke(c.io.in(1).valid, 0)  // Valid input 0
    // What do you think the output will be?
    println(s"valid input 0: in(0).ready=${peek(c.io.in(0).ready)}, in(1).ready=${peek(c.io.in(1).ready)}, out.valid=${peek(c.io.out.valid)}, out.bits=${peek(c.io.out.bits)}")
} }

[info] [0.000] Elaborating design...
[info] [0.007] Done elaborating.
Total FIRRTL Compile Time: 22.3 ms
Total FIRRTL Compile Time: 16.2 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1512597101664
[info] [0.001] Start: in(0).ready=0, in(1).ready=0, out.valid=0, out.bits=99
[info] [0.001] valid input 1: in(0).ready=0, in(1).ready=0, out.valid=1, out.bits=42
[info] [0.002] valid inputs 0 and 1: in(0).ready=0, in(1).ready=0, out.valid=1, out.bits=43
[info] [0.003] valid input 0: in(0).ready=0, in(1).ready=0, out.valid=1, out.bits=43
test cmd20WrapperHelperanonfun1anon1 Success: 0 tests passed in 5 cycles taking 0.006120 seconds
[info] [0.003] RAN 0 CYCLES PASSED


res20: Boolean = true

# Misc Function Blocks
## Bitwise Utilities
### PopCount

### Reverse


## OneHot encoding utilities
### UInt to OneHot
### OneHot to UInt



## Muxes
### OneHot Mux

### Priority Mux




## Counter


---
# You're done!

[Return to the top.](#top)